# Training
This files is meant to train a model to be used as a test model to my metric

This code was heavily inspired on [Official PyTorch MNIST CNN example](https://github.com/pytorch/examples/tree/main/mnist)\
and [ANN Income Prediction](https://github.com/Pranav143/Using-an-ANN-to-predict-income-level-based-on-demographic-data)

By running this file, you should get a file called `mnist_cnn.pt` and a file called `income_model.pt`, which is a torch.jit model, that contains the entire model to be loaded and tested with the metric.\
These files have to be copied into `./tests/data/mnist_cnn.pt` and `./tests/data/income_model.pt` in order to be used by the project

In [1]:
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split

from torch.utils.data import DataLoader
from torch.autograd import Variable
import torch.optim as optim
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import numpy as np
from tqdm import tqdm

MNIST_MODEL_PATH = "mnist_cnn.pt"
INCOME_MODEL_PATH = "income_model.pt"

def exit():
    class StopExecution(Exception):
        def _render_traceback_(self):
            return []
    raise StopExecution

## Model Architecture - Simple CNN model

In [2]:
from model import MNIST_Model, IncomeModel, AdultDataset
from model import get_mnist_dataset_loaders, load_cnn_model

In [3]:
def calculate_metrics(model: MNIST_Model, test_loader: DataLoader) -> tuple[np.ndarray, float]:
    """
    :return confusion_matrix, accuracy:
    """
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()

    confusion_matrix = np.zeros((10, 10), dtype=int)
    total = 0
    with torch.no_grad():
        for images_batch, labels_batch in test_loader:
            images_batch, labels_batch = images_batch.to(device), labels_batch.to(device)
            outputs = model(images_batch)
            _, predicted = torch.max(outputs.data, 1)
            for idx in range(labels_batch.size(0)):
                true_label = int(labels_batch[idx].item())
                pred_label = int(predicted[idx].item())
                confusion_matrix[true_label][pred_label] += 1
            total += labels_batch.size(0)
            
    correct = np.trace(confusion_matrix)    
    accuracy = 100 * correct / total
    return confusion_matrix, accuracy

In [4]:
def train_new_model(train_loader, test_loader, num_epochs=30) -> MNIST_Model:
    model = MNIST_Model()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    try:
        for epoch in range(1, num_epochs + 1):
            epoch_loss = 0
            for images_batch, labels_batch in tqdm(train_loader, desc=f"Epoch {epoch}/{num_epochs}"):
                images_batch, labels_batch = images_batch.to(device), labels_batch.to(device)
                
                pred = model(images_batch)
                loss = loss_fn(pred, labels_batch)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                epoch_loss += loss.item()

            print("Epoch loss:", epoch_loss)
            _, accuracy = calculate_metrics(model, test_loader)
            print(f"Accuracy after epoch {epoch}: {accuracy:.2f}%")
        return model
    except KeyboardInterrupt:
        print("Training interrupted, returning the current model state")
        return model


In [5]:
train_loader, test_loader = get_mnist_dataset_loaders()
model: MNIST_Model | None = load_cnn_model()
if model is not None:
    print("Model already trained on the default path")
    _, accuracy = calculate_metrics(model, test_loader)
    print(f"Previous model accuracy: {accuracy:.2f}%")
    print("Train new model? (y/n): ", end="")
    choice = input().strip().lower()
    if choice == 'n':
        exit()
else:
    print("No model found")

print("Proceeding to training a new model. To interrupt training and potentially save the model weights, press Ctrl+C.")
model = train_new_model(train_loader, test_loader, num_epochs=15)

# Calculate accuracy on test dataset
_, accuracy = calculate_metrics(model, test_loader)
print(f"\nModel accuracy on test dataset: {accuracy:.2f}%")
print("Save model? (y/n): ", end="")
choice = input().strip().lower()
if choice == 'y':
    model.to(torch.device('cpu'))
    model_scripted = torch.jit.script(model)
    torch.jit.save(model_scripted, MNIST_MODEL_PATH)
    print(f"Model saved to {MNIST_MODEL_PATH}")

Error loading model from mnist_cnn.pt: The provided filename mnist_cnn.pt does not exist
No model found
Proceeding to training a new model. To interrupt training and potentially save the model weights, press Ctrl+C.


Epoch 1/15: 100%|██████████| 938/938 [00:09<00:00, 99.14it/s] 


Epoch loss: 183.37453256780282
Accuracy after epoch 1: 98.60%


Epoch 2/15: 100%|██████████| 938/938 [00:09<00:00, 103.74it/s]


Epoch loss: 34.64783460495528
Accuracy after epoch 2: 98.55%


...

Epoch loss: 4.263396198303646
Accuracy after epoch 13: 98.92%


Epoch 14/15: 100%|██████████| 938/938 [00:09<00:00, 103.55it/s]


Epoch loss: 3.968943735520284
Accuracy after epoch 14: 98.61%


Epoch 15/15: 100%|██████████| 938/938 [00:09<00:00, 103.78it/s]


Epoch loss: 2.6423181523926766
Accuracy after epoch 15: 99.04%

Model accuracy on test dataset: 99.04%
Save model? (y/n): Model saved to mnist_cnn.pt


In [6]:
# Caclulate Lipschitz metric
from Lipschitz import measure
if model:
    model.to(torch.device('cpu'))
    model.eval()
    images, labels = next(iter(test_loader))
    images, labels = images.cpu(), labels.cpu()
    scores = measure(model, images, labels)
    print("Local Lipschitz Estimates:", scores)
else:
    print("Model is not loaded, please train a model or make sure the mnist_cnn.pt file exists")

/media/jgaspar/7CCE3FA7CE3F589A1/MCS/1_Semester/ai/a4s-eval/A4S_metric_tests/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Local Lipschitz Estimates: [2.5701136589050293]
Local Lipschitz Estimates: [{'name': 'local_lipschitz_estimate', 'score': 2.5701136589050293, 'time': datetime.datetime(2025, 12, 4, 22, 53, 6, 166873)}]


In [7]:
NUM_EPOCHS = 10
BATCH_SIZE = 64
LEARNING_RATE = 0.001

In [8]:
data = pd.read_csv(r'./data/income/adult.csv')

col_names = data.columns
num_rows = data.shape[0]
data.drop_duplicates(inplace=True)
data.replace('?', np.nan, inplace=True)
data.dropna(inplace=True)

data = data.sample(frac=1).reset_index(drop=True)  # reshuffle dataset and drop new column of index labelling that is made

categorical_features = ['workclass', 'education', 'marital.status', 'occupation', 
                        'relationship', 'race', 'sex', 'native.country', 'income']
for feature in categorical_features:
    label_encoder = LabelEncoder()
    data[feature] = label_encoder.fit_transform(data[feature])

X = data.drop('income', axis=1)
y = data['income']

continuous_features = ['age', 'fnlwgt', 'education.num', 'capital.gain', 'capital.loss', 'hours.per.week']
scaler = StandardScaler()
X[continuous_features] = scaler.fit_transform(X[continuous_features])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)
X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.long)
X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.long)


def load_data(batch_size):
    train_dataset = AdultDataset(X_train_tensor, y_train_tensor)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    
    validation_set = AdultDataset(X_test_tensor, y_test_tensor)
    val_loader = DataLoader(validation_set, batch_size=batch_size, shuffle=True)
    
    return train_loader, val_loader


def load_model(lr):
    global X_train_tensor
    input_dim = X_train_tensor.shape[1]
    model = IncomeModel(input_dim)
    loss_fnc = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    return model, loss_fnc, optimizer


train_loader, val_loader = load_data(BATCH_SIZE)
model, criterion, optimizer = load_model(LEARNING_RATE)

num_correct_pred = 0
num_N_steps = 0
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
for epoch in range(0, NUM_EPOCHS, 1):
    epoch_loss = 0
    model.train()
    for inputs, labels in tqdm(train_loader):
        optimizer.zero_grad()
        inputs = inputs.to(device)  # torch.Size([15, 103])
        labels = labels.to(device)  # torch.Size([15])

        predictions = model(inputs)
        loss = criterion(predictions, labels)
        epoch_loss += loss.item()
        loss.backward()
        optimizer.step()
    print("Epoch loss:", epoch_loss/len(train_loader))
    _, accuracy = calculate_metrics(model, val_loader)
    print(f"Accuracy after epoch {epoch+1}: {accuracy:.2f}%")


model.to(torch.device('cpu'))
model_scripted = torch.jit.script(model)
torch.jit.save(model_scripted, INCOME_MODEL_PATH)
print(f"Model saved to {INCOME_MODEL_PATH}")

100%|██████████| 377/377 [00:00<00:00, 569.95it/s]


Epoch loss: 0.5106229092777566
Accuracy after epoch 1: 80.67%


100%|██████████| 377/377 [00:00<00:00, 552.06it/s]


Epoch loss: 0.42480970100951765
Accuracy after epoch 2: 82.65%


100%|██████████| 377/377 [00:00<00:00, 567.99it/s]


...

Epoch loss: 0.35432067920579835
Accuracy after epoch 8: 84.52%


100%|██████████| 377/377 [00:00<00:00, 583.46it/s]


Epoch loss: 0.35304789549475957
Accuracy after epoch 9: 84.65%


100%|██████████| 377/377 [00:00<00:00, 564.76it/s]


Epoch loss: 0.3537596389058414
Accuracy after epoch 10: 84.46%
Model saved to income_model.pt
